In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

In [3]:
# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")  # Add a user-agent string

# Crawl links

### Crawl links mua bán nhà ở from nhatot.com

In [ ]:
# Initialize the array to store the links
arr = []
start_page = 1
end_page = 1000

# Retry logic
MAX_RETRIES = 3

for page in range(start_page, end_page + 1):
    for attempt in range(MAX_RETRIES):
        try:
            # Initialize WebDriver for each page
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            url = f"https://www.nhatot.com/mua-ban-nha-dat?page={page}"
            driver.get(url)

            # Use explicit wait for better control
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'AdItem_adItem__gDDQT')))

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all the links with the specified class
            a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

            # Append the full link to the array
            for a in a_tags:
                full_link = "https://www.nhatot.com" + a['href']
                arr.append(full_link)

            driver.quit()
            break  # Break if the request was successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt + 1 == MAX_RETRIES:
                print("Max retries reached. Skipping this page.")
            time.sleep(5)  # Wait before retrying

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('muaBanNhaOLinks.csv', index=True)

print("done")

### Crawl links mua bán căn hộ chung cư from nhatot.com

In [ ]:
# Initialize the array to store the links
arr = []
start_page = 1
end_page = 100

# Retry logic
MAX_RETRIES = 3

for page in range(start_page, end_page + 1):
    for attempt in range(MAX_RETRIES):
        try:
            # Initialize WebDriver for each page
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            url = f"https://www.nhatot.com/mua-ban-can-ho-chung-cu-tp-ho-chi-minh?page={page}"
            driver.get(url)

            # Use explicit wait for better control
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'AdItem_adItem__gDDQT')))

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all the links with the specified class
            a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

            # Append the full link to the array
            for a in a_tags:
                full_link = "https://www.nhatot.com" + a['href']
                arr.append(full_link)

            driver.quit()
            break  # Break if the request was successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt + 1 == MAX_RETRIES:
                print("Max retries reached. Skipping this page.")
            time.sleep(5)  # Wait before retrying

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('muaBanCanHoChungCuLinks.csv', index=True)

print("done")

### Crawl links mua bán mặt bằng from nhatot.com

In [7]:
# Initialize the array to store the links
arr = []
start_page = 1
end_page = 200

# Retry logic
MAX_RETRIES = 3

for page in range(start_page, end_page + 1):
    for attempt in range(MAX_RETRIES):
        try:
            # Initialize WebDriver for each page
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            url = f"https://www.nhatot.com/sang-nhuong-van-phong-mat-bang-kinh-doanh-tp-ho-chi-minh?page={page}"
            driver.get(url)

            # Use explicit wait for better control
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'AdItem_adItem__gDDQT')))

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all the links with the specified class
            a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

            # Append the full link to the array
            for a in a_tags:
                full_link = "https://www.nhatot.com" + a['href']
                arr.append(full_link)

            driver.quit()
            break  # Break if the request was successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt + 1 == MAX_RETRIES:
                print("Max retries reached. Skipping this page.")
            time.sleep(5)  # Wait before retrying

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('muaBanMatBangLinks.csv', index=True)

print("done")

Attempt 1 failed: Message: timeout: Timed out receiving message from renderer: 299.079
  (Session info: chrome=128.0.6613.138)
Stacktrace:
	GetHandleVerifier [0x00C0D933+25811]
	(No symbol) [0x00B9E314]
	(No symbol) [0x00A92523]
	(No symbol) [0x00A7E4B5]
	(No symbol) [0x00A7E1E8]
	(No symbol) [0x00A7C5E8]
	(No symbol) [0x00A7CC9D]
	(No symbol) [0x00A8903A]
	(No symbol) [0x00A9D6A5]
	(No symbol) [0x00AA1D06]
	(No symbol) [0x00A7D2CC]
	(No symbol) [0x00A9D563]
	(No symbol) [0x00B14A21]
	(No symbol) [0x00AFAAC6]
	(No symbol) [0x00ACBEFD]
	(No symbol) [0x00ACC8FD]
	GetHandleVerifier [0x00EDF143+2981091]
	GetHandleVerifier [0x00F32FF9+3324825]
	GetHandleVerifier [0x00C9B32F+605903]
	GetHandleVerifier [0x00CA2CBC+637020]
	(No symbol) [0x00BA6F4D]
	(No symbol) [0x00BA3DD8]
	(No symbol) [0x00BA3F75]
	(No symbol) [0x00B96406]
	BaseThreadInitThunk [0x7585FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x775F80CE+286]
	RtlGetAppContainerNamedObjectPath [0x775F809E+238]

done


### Crawl links mua bán đất from nhatot.com

In [ ]:
# Initialize the array to store the links
arr = []
start_page = 1
end_page = 1000

# Retry logic
MAX_RETRIES = 3

for page in range(start_page, end_page + 1):
    for attempt in range(MAX_RETRIES):
        try:
            # Initialize WebDriver for each page
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            url = f"https://www.nhatot.com/mua-ban-dat?page={page}"
            driver.get(url)

            # Use explicit wait for better control
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'AdItem_adItem__gDDQT')))

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all the links with the specified class
            a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

            # Append the full link to the array
            for a in a_tags:
                full_link = "https://www.nhatot.com" + a['href']
                arr.append(full_link)

            driver.quit()
            break  # Break if the request was successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt + 1 == MAX_RETRIES:
                print("Max retries reached. Skipping this page.")
            time.sleep(5)  # Wait before retrying

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('muaBanDatLinks.csv', index=True)

print("done")

### Crawl links cho thuê căn hộ chung cư from nhatot.com

In [ ]:
# Initialize the array to store the links
arr = []
start_page = 1
end_page = 1000

# Retry logic
MAX_RETRIES = 3

for page in range(start_page, end_page + 1):
    for attempt in range(MAX_RETRIES):
        try:
            # Initialize WebDriver for each page
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            url = f"https://www.nhatot.com/thue-can-ho-chung-cu?page={page}"
            driver.get(url)

            # Use explicit wait for better control
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'AdItem_adItem__gDDQT')))

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all the links with the specified class
            a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

            # Append the full link to the array
            for a in a_tags:
                full_link = "https://www.nhatot.com" + a['href']
                arr.append(full_link)

            driver.quit()
            break  # Break if the request was successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt + 1 == MAX_RETRIES:
                print("Max retries reached. Skipping this page.")
            time.sleep(5)  # Wait before retrying

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('choThueCanHoChungCuLinks.csv', index=True)

print("done")

### Crawl links cho thuê nhà ở from nhatot.com

In [ ]:
# Initialize the array to store the links
arr = []
start_page = 1
end_page = 1000

# Retry logic
MAX_RETRIES = 3

for page in range(start_page, end_page + 1):
    for attempt in range(MAX_RETRIES):
        try:
            # Initialize WebDriver for each page
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            url = f"https://www.nhatot.com/thue-nha-dat?page={page}"
            driver.get(url)

            # Use explicit wait for better control
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'AdItem_adItem__gDDQT')))

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all the links with the specified class
            a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

            # Append the full link to the array
            for a in a_tags:
                full_link = "https://www.nhatot.com" + a['href']
                arr.append(full_link)

            driver.quit()
            break  # Break if the request was successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt + 1 == MAX_RETRIES:
                print("Max retries reached. Skipping this page.")
            time.sleep(5)  # Wait before retrying

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('choThueNhaOLinks.csv', index=True)

print("done")

### Crawl links cho thuê mặt bằng from nhatot.com

In [ ]:
# Initialize the array to store the links
arr = []
start_page = 1
end_page = 1000

# Retry logic
MAX_RETRIES = 3

for page in range(start_page, end_page + 1):
    for attempt in range(MAX_RETRIES):
        try:
            # Initialize WebDriver for each page
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            url = f"https://www.nhatot.com/thue-van-phong-mat-bang-kinh-doanh?page={page}"
            driver.get(url)

            # Use explicit wait for better control
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'AdItem_adItem__gDDQT')))

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all the links with the specified class
            a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

            # Append the full link to the array
            for a in a_tags:
                full_link = "https://www.nhatot.com" + a['href']
                arr.append(full_link)

            driver.quit()
            break  # Break if the request was successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt + 1 == MAX_RETRIES:
                print("Max retries reached. Skipping this page.")
            time.sleep(5)  # Wait before retrying

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('choThueMatBangLinks.csv', index=True)

print("done")

### Crawl links cho thuê đất from nhatot.com

In [ ]:
# Initialize the array to store the links
arr = []
start_page = 1
end_page = 1000

# Retry logic
MAX_RETRIES = 3

for page in range(start_page, end_page + 1):
    for attempt in range(MAX_RETRIES):
        try:
            # Initialize WebDriver for each page
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            url = f"https://www.nhatot.com/thue-dat?page={page}"
            driver.get(url)

            # Use explicit wait for better control
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'AdItem_adItem__gDDQT')))

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all the links with the specified class
            a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

            # Append the full link to the array
            for a in a_tags:
                full_link = "https://www.nhatot.com" + a['href']
                arr.append(full_link)

            driver.quit()
            break  # Break if the request was successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt + 1 == MAX_RETRIES:
                print("Max retries reached. Skipping this page.")
            time.sleep(5)  # Wait before retrying

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('choThueDatLinks.csv', index=True)

print("done")

### Crawl links cho thuê phòng trọ from nhatot.com

In [ ]:
# Initialize the array to store the links
arr = []
start_page = 1
end_page = 1000

# Retry logic
MAX_RETRIES = 3

for page in range(start_page, end_page + 1):
    for attempt in range(MAX_RETRIES):
        try:
            # Initialize WebDriver for each page
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            url = f"https://www.nhatot.com/thue-phong-tro?page={page}"
            driver.get(url)

            # Use explicit wait for better control
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'AdItem_adItem__gDDQT')))

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all the links with the specified class
            a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

            # Append the full link to the array
            for a in a_tags:
                full_link = "https://www.nhatot.com" + a['href']
                arr.append(full_link)

            driver.quit()
            break  # Break if the request was successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt + 1 == MAX_RETRIES:
                print("Max retries reached. Skipping this page.")
            time.sleep(5)  # Wait before retrying

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('choThuePhongTroLinks.csv', index=True)

print("done")

# Test lấy dữ liệu

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Truy cập vào trang web
url = 'https://www.nhatot.com/mua-ban-nha-dat-thanh-pho-bien-hoa-dong-nai/119225036.htm#px=SR-stickyad-[PO-1][PL-top]'
driver.get(url)

# Đợi một chút để trang tải hết nội dung
time.sleep(5)

# Lấy nội dung của trang
page_source = driver.page_source

# Đóng trình duyệt
driver.quit()

# Phân tích cú pháp HTML với BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

price = soup.find("b", class_='pyhk1dv').text
print(price)

address = soup.find_all("span", class_="bwq0cbs")
print(address[4].text)

price_m2 = soup.find("span", itemprop="price_m2").text
rooms = soup.find("span", itemprop="rooms").text
toilets = soup.find("span", itemprop="toilets").text
direction = soup.find("span", itemprop="direction").text
floors = soup.find("span", itemprop="floors").text
property_legal_document = soup.find("span", itemprop="property_legal_document").text
house_type = soup.find("span", itemprop="house_type").text
furnishing_sell = soup.find("span", itemprop="furnishing_sell").text
width = soup.find("span", itemprop="width").text
length = soup.find("span", itemprop="length").text
living_size = soup.find("span", itemprop="living_size").text
size = soup.find("span", itemprop="size").text




# Lấy chi tiết data

### Lấy chi tiết data mua bán nhà ở

In [ ]:
from tenacity import retry, wait_fixed, stop_after_attempt

# Function to extract data from a single page
@retry(wait=wait_fixed(2), stop=stop_after_attempt(3))  # Wait 2 seconds between retries, retry 3 times
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(url)

        # Wait until price element is available or timeout after 10 seconds
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pyhk1dv')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        try:
            price = soup.find("b", class_='pyhk1dv').text
        except AttributeError:
            price = None

        try:
            address = soup.find_all("span", class_="bwq0cbs")[4].text
        except (IndexError, AttributeError):
            address = None

        try:
            price_m2 = soup.find("span", itemprop="price_m2").text
        except AttributeError:
            price_m2 = None

        try:
            rooms = soup.find("span", itemprop="rooms").text
        except AttributeError:
            rooms = None

        try:
            toilets = soup.find("span", itemprop="toilets").text
        except AttributeError:
            toilets = None

        try:
            direction = soup.find("span", itemprop="direction").text
        except AttributeError:
            direction = None

        try:
            floors = soup.find("span", itemprop="floors").text
        except AttributeError:
            floors = None

        try:
            property_legal_document = soup.find("span", itemprop="property_legal_document").text
        except AttributeError:
            property_legal_document = None

        try:
            house_type = soup.find("span", itemprop="house_type").text
        except AttributeError:
            house_type = None

        try:
            furnishing_sell = soup.find("span", itemprop="furnishing_sell").text
        except AttributeError:
            furnishing_sell = None

        try:
            width = soup.find("span", itemprop="width").text
        except AttributeError:
            width = None

        try:
            length = soup.find("span", itemprop="length").text
        except AttributeError:
            length = None

        try:
            living_size = soup.find("span", itemprop="living_size").text
        except AttributeError:
            living_size = None

        try:
            size = soup.find("span", itemprop="size").text
        except AttributeError:
            size = None
        
        driver.quit()

        return {
            "price": price,
            "address": address,
            "price_m2": price_m2,
            "rooms": rooms,
            "toilets": toilets,
            "direction": direction,
            "floors": floors,
            "property_legal_document": property_legal_document,
            "house_type": house_type,
            "furnishing_sell": furnishing_sell,
            "width": width,
            "length": length,
            "living_size": living_size,
            "size": size
        }
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        driver.quit()
        return None

# Function to save data to CSV
def save_to_csv(data_list, filename='muaBanNhaDat3.csv'):
    if not data_list:
        print("No data to save.")
        return
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    # Replace this with your actual DataFrame or list of URLs
    frame = pd.read_csv('nhaDatLinks1.csv')  # Assuming you already have the 'Links' data in CSV
    all_data = []
    i = 0
    for url in frame['Links']:
        data = extract_data(url)
        i+=1
        if data:
            all_data.append(data)
        print(f"Processed URL {i}: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()

In [ ]:
frame1 = pd.read_csv('muaBanNhaDat.csv')
frame1.head()

### Lấy chi tiết data mua bán căn hộ chung cư

In [6]:
# Function to extract data from a single page
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(url)

        # Wait until price element is available or timeout after 10 seconds
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pyhk1dv')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        try:
            price = soup.find("b", class_='pyhk1dv').text
        except AttributeError:
            price = None

        try:
            address = soup.find_all("span", class_="bwq0cbs")[5].text
        except (IndexError, AttributeError):
            address = None

        try:
            price_m2 = soup.find("span", itemprop="price_m2").text
        except AttributeError:
            price_m2 = None

        try:
            rooms = soup.find("span", itemprop="rooms").text
        except AttributeError:
            rooms = None

        try:
            toilets = soup.find("span", itemprop="toilets").text
        except AttributeError:
            toilets = None

        try:
            direction = soup.find("span", itemprop="direction").text
        except AttributeError:
            direction = None

        try:
            property_status = soup.find("span", itemprop="property_status").text
        except AttributeError:
            property_status = None
            
        try:
            balconydirection = soup.find("span", itemprop="balconydirection").text
        except AttributeError:
            balconydirection = None
            
        try:
            property_legal_document = soup.find("span", itemprop="property_legal_document").text
        except AttributeError:
            property_legal_document = None

        try:
            apartment_type = soup.find("span", itemprop="apartment_type").text
        except AttributeError:
            apartment_type = None

        try:
            furnishing_sell = soup.find("span", itemprop="furnishing_sell").text
        except AttributeError:
            furnishing_sell = None

        try:
            size = soup.find("span", itemprop="size").text
        except AttributeError:
            size = None
            
        try:
            apartment_feature = soup.find("span", itemprop="apartment_feature").text
        except AttributeError:
            apartment_feature = None
        
        driver.quit()

        return {
            "price": price,
            "address": address,
            "price_m2": price_m2,
            "rooms": rooms,
            "toilets": toilets,
            "direction": direction,
            "property_status": property_status,
            "balconydirection": balconydirection,
            "property_legal_document": property_legal_document,
            "apartment_type": apartment_type,
            "furnishing_sell": furnishing_sell,
            "size": size,
            "apartment_feature": apartment_feature
        }
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        driver.quit()
        return None

# Function to save data to CSV
def save_to_csv(data_list, filename='muaBanCanHo1.csv'):
    if not data_list:
        print("No data to save.")
        return
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    # Replace this with your actual DataFrame or list of URLs
    frame = pd.read_csv('canHoLinks.csv')  # Assuming you already have the 'Links' data in CSV
    all_data = []
    i = 0
    for url in frame['Links']:
        data = extract_data(url)
        i+=1
        if data:
            all_data.append(data)
        print(f"Processed URL {i}: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()

Processed URL 1: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-12-tp-ho-chi-minh/118579402.htm
Processed URL 2: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-12-tp-ho-chi-minh/118578440.htm
Processed URL 3: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-12-tp-ho-chi-minh/118578263.htm
Processed URL 4: https://www.nhatot.com/mua-ban-can-ho-chung-cu-huyen-binh-chanh-tp-ho-chi-minh/112894919.htm
Processed URL 5: https://www.nhatot.com/mua-ban-can-ho-chung-cu-huyen-binh-chanh-tp-ho-chi-minh/117044850.htm
Processed URL 6: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-binh-tan-tp-ho-chi-minh/119260810.htm
Processed URL 7: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-binh-thanh-tp-ho-chi-minh/114992122.htm
Processed URL 8: https://www.nhatot.com/mua-ban-can-ho-chung-cu-huyen-nha-be-tp-ho-chi-minh/118570764.htm
Processed URL 9: https://www.nhatot.com/mua-ban-can-ho-chung-cu-thanh-pho-thu-duc-tp-ho-chi-minh/118199275.htm
Processed URL 10: https://www.nhatot.com/mua

### Lấy chi tiết data mua bán mặt bằng

In [ ]:
# Function to extract data from a single page
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(url)

        # Wait until price element is available or timeout after 10 seconds
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pyhk1dv')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        try:
            price = soup.find("b", class_='pyhk1dv').text
        except AttributeError:
            price = None

        try:
            address = soup.find_all("span", class_="bwq0cbs")[3].text
        except (IndexError, AttributeError):
            address = None

        try:
            price_m2 = soup.find("span", itemprop="price_m2").text
        except AttributeError:
            price_m2 = None

        try:
            direction = soup.find("span", itemprop="direction").text
        except AttributeError:
            direction = None
            
        try:
            property_legal_document = soup.find("span", itemprop="property_legal_document").text
        except AttributeError:
            property_legal_document = None

        try:
            commercial_type = soup.find("span", itemprop="commercial_type").text
        except AttributeError:
            commercial_type = None

        try:
            furnishing_sell = soup.find("span", itemprop="furnishing_sell").text
        except AttributeError:
            furnishing_sell = None

        try:
            size = soup.find("span", itemprop="size").text
        except AttributeError:
            size = None
        
        driver.quit()

        return {
            "price": price,
            "address": address,
            "price_m2": price_m2,
            "direction": direction,
            "property_legal_document": property_legal_document,
            "commercial_type": commercial_type,
            "furnishing_sell": furnishing_sell,
            "size": size,
        }
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        driver.quit()
        return None

# Function to save data to CSV
def save_to_csv(data_list, filename='matBang.csv'):
    if not data_list:
        print("No data to save.")
        return
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    # Replace this with your actual DataFrame or list of URLs
    frame = pd.read_csv('matBangLinks.csv')  # Assuming you already have the 'Links' data in CSV
    all_data = []
    i = 0
    for url in frame['Links']:
        data = extract_data(url)
        i+=1
        if data:
            all_data.append(data)
        print(f"Processed URL {i}: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()

### Lấy chi tiết data mua bán đất

In [ ]:
# Function to extract data from a single page
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(url)

        # Wait until price element is available or timeout after 10 seconds
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pyhk1dv')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        try:
            price = soup.find("b", class_='pyhk1dv').text
        except AttributeError:
            price = None

        try:
            address = soup.find_all("span", class_="bwq0cbs")[3].text
        except (IndexError, AttributeError):
            address = None

        try:
            price_m2 = soup.find("span", itemprop="price_m2").text
        except AttributeError:
            price_m2 = None

        try:
            direction = soup.find("span", itemprop="direction").text
        except AttributeError:
            direction = None
            
        try:
            property_legal_document = soup.find("span", itemprop="property_legal_document").text
        except AttributeError:
            property_legal_document = None

        try:
            land_type = soup.find("span", itemprop="land_type").text
        except AttributeError:
            land_type = None

        try:
            pty_characteristics = soup.find("span", itemprop="pty_characteristics").text
        except AttributeError:
            pty_characteristics = None

        try:
            size = soup.find("span", itemprop="size").text
        except AttributeError:
            size = None
        
        try:
            width = soup.find("span", itemprop="width").text
        except AttributeError:
            width = None

        try:
            length = soup.find("span", itemprop="length").text
        except AttributeError:
            length = None
        
        driver.quit()

        return {
            "price": price,
            "address": address,
            "price_m2": price_m2,
            "direction": direction,
            "property_legal_document": property_legal_document,
            "land_type": land_type,
            "pty_characteristics": pty_characteristics,
            "size": size,
            "width": width,
            "length": length,
        }
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        driver.quit()
        return None

# Function to save data to CSV
def save_to_csv(data_list, filename='raw_data.csv'):
    if not data_list:
        print("No data to save.")
        return
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    # Replace this with your actual DataFrame or list of URLs
    frame = pd.read_csv('muaBanDatLinks.csv')  # Assuming you already have the 'Links' data in CSV
    all_data = []
    
    for url in frame['Links']:
        data = extract_data(url)
        if data:
            all_data.append(data)
        print(f"Processed URL: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()

### Lấy chi tiết data cho thuê căn hộ chung cư

In [ ]:
# Function to extract data from a single page
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(url)

        # Wait until price element is available or timeout after 10 seconds
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pyhk1dv')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        try:
            price = soup.find("b", class_='pyhk1dv').text
        except AttributeError:
            price = None

        try:
            address = soup.find_all("span", class_="bwq0cbs")[0].text
        except (IndexError, AttributeError):
            address = None

        try:
            ad_type = soup.find("span", itemprop="ad_type").text
        except AttributeError:
            ad_type = None
            
        try:
            rooms = soup.find("span", itemprop="rooms").text
        except AttributeError:
            rooms = None
            
        try:
            toilets = soup.find("span", itemprop="toilets").text
        except AttributeError:
            toilets = None

        try:
            direction = soup.find("span", itemprop="direction").text
        except AttributeError:
            direction = None
            
        try:
            balconydirection = soup.find("span", itemprop="balconydirection").text
        except AttributeError:
            balconydirection = None
            
        try:
            property_legal_document = soup.find("span", itemprop="property_legal_document").text
        except AttributeError:
            property_legal_document = None

        try:
            apartment_type = soup.find("span", itemprop="apartment_type").text
        except AttributeError:
            apartment_type = None

        try:
            furnishing_sell = soup.find("span", itemprop="furnishing_sell").text
        except AttributeError:
            furnishing_sell = None

        try:
            size = soup.find("span", itemprop="size").text
        except AttributeError:
            size = None
        
        try:
            deposit = soup.find("span", itemprop="deposit").text
        except AttributeError:
            deposit = None
        
        driver.quit()

        return {
            "price": price,
            "address": address,
            "ad_type": ad_type,
            "rooms": rooms,
            "toilets": toilets,
            "direction": direction,
            "balconydirection": balconydirection,
            "property_legal_document": property_legal_document,
            "apartment_type": apartment_type,
            "furnishing_sell": furnishing_sell,
            "deposit": deposit,
            "size": size,
        }
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        driver.quit()
        return None

# Function to save data to CSV
def save_to_csv(data_list, filename='choThueCanHoChungCu.csv'):
    if not data_list:
        print("No data to save.")
        return
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    # Replace this with your actual DataFrame or list of URLs
    frame = pd.read_csv('choThueCanHoChungCuLinks.csv')  # Assuming you already have the 'Links' data in CSV
    all_data = []
    
    for url in frame['Links']:
        data = extract_data(url)
        if data:
            all_data.append(data)
        print(f"Processed URL: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()

### Lấy chi tiết data cho thuê nhà ở

In [ ]:
# Function to extract data from a single page
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(url)

        # Wait until price element is available or timeout after 10 seconds
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pyhk1dv')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        try:
            price = soup.find("b", class_='pyhk1dv').text
        except AttributeError:
            price = None

        try:
            address = soup.find_all("span", class_="bwq0cbs")[0].text
        except (IndexError, AttributeError):
            address = None

        try:
            ad_type = soup.find("span", itemprop="ad_type").text
        except AttributeError:
            ad_type = None

        try:
            rooms = soup.find("span", itemprop="rooms").text
        except AttributeError:
            rooms = None
            
        try:
            toilets = soup.find("span", itemprop="toilets").text
        except AttributeError:
            toilets = None

        try:
            direction = soup.find("span", itemprop="direction").text
        except AttributeError:
            direction = None
            
        try:
            floors = soup.find("span", itemprop="floors").text
        except AttributeError:
            floors = None
            
        try:
            property_legal_document = soup.find("span", itemprop="property_legal_document").text
        except AttributeError:
            property_legal_document = None

        try:
            house_type = soup.find("span", itemprop="house_type").text
        except AttributeError:
            house_type = None

        try:
            furnishing_sell = soup.find("span", itemprop="furnishing_sell").text
        except AttributeError:
            furnishing_sell = None

        try:
            size = soup.find("span", itemprop="size").text
        except AttributeError:
            size = None
        
        try:
            deposit = soup.find("span", itemprop="deposit").text
        except AttributeError:
            deposit = None
            
        try:
            living_size = soup.find("span", itemprop="living_size").text
        except AttributeError:
            living_size = None
            
        try:
            pty_characteristics = soup.find("span", itemprop="pty_characteristics").text
        except AttributeError:
            pty_characteristics = None
            
        try:
            width = soup.find("span", itemprop="width").text
        except AttributeError:
            width = None
            
        try:
            length = soup.find("span", itemprop="length").text
        except AttributeError:
            length = None
        
        driver.quit()

        return {
            "price": price,
            "address": address,
            "ad_type": ad_type,
            "rooms": rooms,
            "toilets": toilets,
            "direction": direction,
            "floors": floors,
            "property_legal_document": property_legal_document,
            "house_type": house_type,
            "furnishing_sell": furnishing_sell,
            "deposit": deposit,
            "size": size,
            "living_size": living_size,
            "pty_characteristics": pty_characteristics,
            "width": width,
            "length": length,
        }
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        driver.quit()
        return None

# Function to save data to CSV
def save_to_csv(data_list, filename='choThueNhaO.csv'):
    if not data_list:
        print("No data to save.")
        return
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    # Replace this with your actual DataFrame or list of URLs
    frame = pd.read_csv('choThueNhaOLinks.csv')  # Assuming you already have the 'Links' data in CSV
    all_data = []
    
    for url in frame['Links']:
        data = extract_data(url)
        if data:
            all_data.append(data)
        print(f"Processed URL: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()

### Lấy chi tiết data cho thuê mặt bằng

In [ ]:
# Function to extract data from a single page
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(url)

        # Wait until price element is available or timeout after 10 seconds
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pyhk1dv')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        try:
            price = soup.find("b", class_='pyhk1dv').text
        except AttributeError:
            price = None

        try:
            address = soup.find_all("span", class_="bwq0cbs")[0].text
        except (IndexError, AttributeError):
            address = None

        try:
            ad_type = soup.find("span", itemprop="ad_type").text
        except AttributeError:
            ad_type = None

        try:
            direction = soup.find("span", itemprop="direction").text
        except AttributeError:
            direction = None
            
        try:
            property_legal_document = soup.find("span", itemprop="property_legal_document").text
        except AttributeError:
            property_legal_document = None

        try:
            commercial_type = soup.find("span", itemprop="commercial_type").text
        except AttributeError:
            commercial_type = None

        try:
            furnishing_sell = soup.find("span", itemprop="furnishing_sell").text
        except AttributeError:
            furnishing_sell = None

        try:
            size = soup.find("span", itemprop="size").text
        except AttributeError:
            size = None
        
        try:
            deposit = soup.find("span", itemprop="deposit").text
        except AttributeError:
            deposit = None
        
        driver.quit()

        return {
            "price": price,
            "address": address,
            "ad_type": ad_type,
            "direction": direction,
            "property_legal_document": property_legal_document,
            "commercial_type": commercial_type,
            "furnishing_sell": furnishing_sell,
            "deposit": deposit,
            "size": size,
        }
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        driver.quit()
        return None

# Function to save data to CSV
def save_to_csv(data_list, filename='choThueMatBang.csv'):
    if not data_list:
        print("No data to save.")
        return
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    # Replace this with your actual DataFrame or list of URLs
    frame = pd.read_csv('choThueMatBangLinks.csv')  # Assuming you already have the 'Links' data in CSV
    all_data = []
    
    for url in frame['Links']:
        data = extract_data(url)
        if data:
            all_data.append(data)
        print(f"Processed URL: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()

### Lấy chi tiết data cho thuê đất

In [ ]:
# Function to extract data from a single page
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(url)

        # Wait until price element is available or timeout after 10 seconds
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pyhk1dv')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        try:
            price = soup.find("b", class_='pyhk1dv').text
        except AttributeError:
            price = None

        try:
            address = soup.find_all("span", class_="bwq0cbs")[0].text
        except (IndexError, AttributeError):
            address = None

        try:
            ad_type = soup.find("span", itemprop="ad_type").text
        except AttributeError:
            ad_type = None

        try:
            direction = soup.find("span", itemprop="direction").text
        except AttributeError:
            direction = None
            
        try:
            property_legal_document = soup.find("span", itemprop="property_legal_document").text
        except AttributeError:
            property_legal_document = None

        try:
            land_type = soup.find("span", itemprop="land_type").text
        except AttributeError:
            land_type = None

        try:
            width = soup.find("span", itemprop="width").text
        except AttributeError:
            width = None
            
        try:
            length = soup.find("span", itemprop="length").text
        except AttributeError:
            length = None

        try:
            size = soup.find("span", itemprop="size").text
        except AttributeError:
            size = None
        
        try:
            deposit = soup.find("span", itemprop="deposit").text
        except AttributeError:
            deposit = None
        
        driver.quit()

        return {
            "price": price,
            "address": address,
            "ad_type": ad_type,
            "direction": direction,
            "property_legal_document": property_legal_document,
            "land_type": land_type,
            "width": width,
            "length": length,
            "deposit": deposit,
            "size": size,
        }
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        driver.quit()
        return None

# Function to save data to CSV
def save_to_csv(data_list, filename='choThueDat.csv'):
    if not data_list:
        print("No data to save.")
        return
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    # Replace this with your actual DataFrame or list of URLs
    frame = pd.read_csv('choThueDatLinks.csv')  # Assuming you already have the 'Links' data in CSV
    all_data = []
    
    for url in frame['Links']:
        data = extract_data(url)
        if data:
            all_data.append(data)
        print(f"Processed URL: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()

### Lấy chi tiết data cho thuê phòng trọ

In [ ]:
# Function to extract data from a single page
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(url)

        # Wait until price element is available or timeout after 10 seconds
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pyhk1dv')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        try:
            price = soup.find("b", class_='pyhk1dv').text
        except AttributeError:
            price = None

        try:
            address = soup.find_all("span", class_="bwq0cbs")[0].text
        except (IndexError, AttributeError):
            address = None

        try:
            ad_type = soup.find("span", itemprop="ad_type").text
        except AttributeError:
            ad_type = None

        try:
            furnishing_rent = soup.find("span", itemprop="furnishing_rent").text
        except AttributeError:
            furnishing_rent = None
            
        try:
            size = soup.find("span", itemprop="size").text
        except AttributeError:
            size = None
        
        try:
            deposit = soup.find("span", itemprop="deposit").text
        except AttributeError:
            deposit = None
        
        driver.quit()

        return {
            "price": price,
            "address": address,
            "ad_type": ad_type,
            "furnishing_rent": furnishing_rent, 
            "deposit": deposit,
            "size": size,
        }
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        driver.quit()
        return None

# Function to save data to CSV
def save_to_csv(data_list, filename='choThuePhongTro.csv'):
    if not data_list:
        print("No data to save.")
        return
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    # Replace this with your actual DataFrame or list of URLs
    frame = pd.read_csv('choThuePhongTroLinks.csv')  # Assuming you already have the 'Links' data in CSV
    all_data = []
    
    for url in frame['Links']:
        data = extract_data(url)
        if data:
            all_data.append(data)
        print(f"Processed URL: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()